# Using FastAPI to Run FastKafka Application

When deploying a FastKafka application, the default approach is to utilize the [`fastkafka run`](/docs/cli/fastkafka#fastkafka-run) CLI command. This command allows you to launch your FastKafka application as a standalone service. However, if you already have a FastAPI application in place and wish to run FastKafka application alongside it, you have an alternative option.

FastKafka provides a method called `FastKafka.fastapi_lifespan` that leverages [FastAPI's lifespan](https://fastapi.tiangolo.com/advanced/events/#lifespan-events) feature. This method allows you to run your FastKafka application together with your existing FastAPI app, seamlessly integrating their functionalities. By using the `FastKafka.fastapi_lifespan` method, you can start the FastKafka application within the same process as the FastAPI app.

The `FastKafka.fastapi_lifespan` method ensures that both FastAPI and FastKafka are initialized and start working simultaneously. This approach enables the execution of Kafka-related tasks, such as producing and consuming messages, while also handling HTTP requests through FastAPI's routes.

By combining FastAPI and FastKafka in this manner, you can build a comprehensive application that harnesses the power of both frameworks. Whether you require real-time messaging capabilities or traditional HTTP endpoints, this approach allows you to leverage the strengths of FastAPI and FastKafka within a single deployment setup.

## Prerequisites


1. A basic knowledge of `FastKafka` is needed to proceed with this guide. If you are not familiar with `FastKafka`, please go through the [tutorial](/docs#tutorial) first.
2. `FastKafka` and `FastAPI` libraries needs to be installed.

This guide will provide a step-by-step explanation, taking you through each stage individually, before combining all the components in the final section for a comprehensive understanding of the process.

## 1. Basic FastKafka app

In this step, we will begin by creating a simple FastKafka application.

In [ ]:
from pydantic import BaseModel, Field, NonNegativeFloat
from typing import *

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Greetings",
    kafka_brokers=kafka_brokers,
)


class TestMsg(BaseModel):
    msg: str = Field(...)


@kafka_app.consumes()
async def on_names(msg: TestMsg):
    await to_greetings(TestMsg(msg=f"Hello {msg.msg}"))


@kafka_app.produces()
async def to_greetings(greeting: TestMsg) -> TestMsg:
    return greeting

In the above example, we consume messages from a topic called `names`, we prepend "Hello" to the message, and send it back to another topic called `greetings`.

We now have a simple `FastKafka` app to produce and consume from two topics.

## 2. Using fastapi_lifespan method

In this step of the guide, we will explore the integration of a FastKafka application with a FastAPI application using the `FastKafka.fastapi_lifespan` method. 
The `FastKafka.fastapi_lifespan` method is a feature provided by FastKafka, which allows you to seamlessly integrate a FastKafka application with a FastAPI application by leveraging FastAPI's lifespan feature.

In [ ]:
from fastapi import FastAPI

fastapi_app = FastAPI(lifespan=kafka_app.fastapi_lifespan(kafka_broker_name="localhost"))


@fastapi_app.get("/hello")
async def hello():
    return {"msg": "hello there"}

In the above example, a new instance of the `FastAPI` app is created, and when the app is started using uvicorn, it also runs the `FastKafka` application concurrently.

## Putting it all together

Let's put the above code together and write it in a file called `fast_apps.py`.

In [ ]:
# | echo: false
from IPython.display import Markdown

kafka_app_source = """
from pydantic import BaseModel, Field, NonNegativeFloat
from typing import *

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Greetings",
    kafka_brokers=kafka_brokers,
)


class TestMsg(BaseModel):
    msg: str = Field(...)


@kafka_app.consumes()
async def on_names(msg: TestMsg):
    await to_greetings(TestMsg(msg=f"Hello {msg.msg}"))


@kafka_app.produces()
async def to_greetings(greeting: TestMsg) -> TestMsg:
    return greeting


from fastapi import FastAPI

fastapi_app = FastAPI(lifespan=kafka_app.fastapi_lifespan(kafka_broker_name="localhost"))

@fastapi_app.get("/hello")
async def hello():
    return {"msg": "hello there"}
"""

with open("fast_apps.py", "w") as source:
    source.write(kafka_app_source)

Markdown(
    f"""
```python
# content of the "fast_apps.py" file
{kafka_app_source}
```
"""
)


```python
# content of the "fast_apps.py" file

from pydantic import BaseModel, Field, NonNegativeFloat
from typing import *

from fastkafka import FastKafka

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "production": {
        "url": "kafka.airt.ai",
        "description": "production kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "plain"},
    },
}

kafka_app = FastKafka(
    title="Greetings",
    kafka_brokers=kafka_brokers,
)


class TestMsg(BaseModel):
    msg: str = Field(...)


@kafka_app.consumes()
async def on_names(msg: TestMsg):
    await to_greetings(TestMsg(msg=f"Hello {msg.msg}"))


@kafka_app.produces()
async def to_greetings(greeting: TestMsg) -> TestMsg:
    return greeting


from fastapi import FastAPI

fastapi_app = FastAPI(lifespan=kafka_app.fastapi_lifespan("localhost"))

@fastapi_app.get("/hello")
async def hello():
    return {"msg": "hello there"}

```


Finally, you can run the FastAPI application using a web server of your choice, such as Uvicorn or Hypercorn by running the below command:
```cmd
uvicorn fast_apps:fastapi_app --host=0.0.0.0 --port=8080
```

In [ ]:
# | hide
from fastkafka.testing import Tester
from fastkafka._server import run_in_process
import uvicorn
from fastkafka import FastKafka
import asyncio
import requests


def run_uvicorn():
    uvicorn.run(
        "fast_apps:fastapi_app",
        host="0.0.0.0",
        port=8080,
        reload=False,
        log_level="debug",
        workers=1,
    )


app_for_tester = FastKafka(
    kafka_brokers=dict(localhost=dict(url="localhost", port=9092)),
    group_id="app_for_tester_group",
)


class TestMsg(BaseModel):
    msg: str = Field(...)


@app_for_tester.consumes(topic="names")
async def on_app_for_tester_names(msg: TestMsg):
    await to_app_for_tester_greetings(TestMsg(msg=f"Hello {msg.msg}"))


@app_for_tester.produces(topic="greetings")
async def to_app_for_tester_greetings(greeting: TestMsg) -> TestMsg:
    return greeting


async with Tester(app_for_tester).using_local_kafka(listener_port=9092) as tester:
    with run_in_process(run_uvicorn) as p:
        await asyncio.sleep(3)
        res = requests.get("http://127.0.0.1:8080/hello")
        assert res.ok

        await tester.to_names(TestMsg(msg=f"signal 10"))
        await asyncio.sleep(3)
        assert (
            tester.mocks.on_greetings.call_count == 2
        ), tester.mocks.on_greetings.call_count

    p.close()

print("ok")

23-05-19 12:44:13.150 [INFO] fastkafka._components.test_dependencies: Java is already installed.
23-05-19 12:44:13.151 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-19 12:44:13.151 [INFO] fastkafka._components.test_dependencies: Kafka is installed.
23-05-19 12:44:13.152 [INFO] fastkafka._components.test_dependencies: But not exported to PATH, exporting...
23-05-19 12:44:13.153 [INFO] fastkafka._testing.apache_kafka_broker: Starting zookeeper...
23-05-19 12:44:13.767 [INFO] fastkafka._testing.apache_kafka_broker: Starting kafka...
23-05-19 12:44:15.435 [INFO] fastkafka._testing.apache_kafka_broker: Local Kafka broker up and running on 127.0.0.1:9092
23-05-19 12:44:16.894 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.0.0.1:9092'}'
23-05-19 12:44:16.908 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': '127.

INFO:     Started server process [574015]
INFO:     Waiting for application startup.


23-05-19 12:44:19.946 [INFO] fastkafka._application.app: set_kafka_broker() : Setting bootstrap_servers value to 'localhost:9092'
23-05-19 12:44:19.947 [INFO] fastkafka._application.app: _create_producer() : created producer using the config: '{'bootstrap_servers': 'localhost:9092'}'
23-05-19 12:44:19.953 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() starting...
23-05-19 12:44:19.954 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer created using the following parameters: {'bootstrap_servers': 'localhost:9092', 'auto_offset_reset': 'earliest', 'max_poll_records': 100}


INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8080 (Press CTRL+C to quit)


23-05-19 12:44:19.961 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer started.
23-05-19 12:44:19.961 [INFO] aiokafka.consumer.subscription_state: Updating subscribed topics to: frozenset({'names'})
23-05-19 12:44:19.962 [INFO] aiokafka.consumer.consumer: Subscribed to topic(s): {'names'}
23-05-19 12:44:19.963 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer subscribed.
23-05-19 12:44:19.966 [INFO] aiokafka.consumer.group_coordinator: Metadata for topic has changed from {} to {'names': 1}. 
INFO:     127.0.0.1:56272 - "GET /hello HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.


23-05-19 12:44:26.148 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-05-19 12:44:26.149 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.


INFO:     Application shutdown complete.
INFO:     Finished server process [574015]


23-05-19 12:44:26.157 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-05-19 12:44:26.157 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
23-05-19 12:44:26.266 [INFO] aiokafka.consumer.group_coordinator: LeaveGroup request succeeded
23-05-19 12:44:26.268 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop(): Consumer stopped.
23-05-19 12:44:26.268 [INFO] fastkafka._components.aiokafka_consumer_loop: aiokafka_consumer_loop() finished.
23-05-19 12:44:26.269 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 572846...
23-05-19 12:44:27.853 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Process 572846 terminated.
23-05-19 12:44:27.854 [INFO] fastkafka._components._subprocess: terminate_asyncio_process(): Terminating the process 572474...
23-05-19 12:44:29.183 [INFO] fastkafka._components._subprocess: terminate